In [2]:
pip install pyscipopt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyscipopt import Model, quicksum, multidict
import numpy as np
import pandas as pd
import geopandas as gpd

In [4]:
# adapted from https://scipbook.readthedocs.io/en/latest/flp.html
def flp(I,J,d,M,c,existing_sites=None):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    
    if existing_sites:
        for j in existing_sites:
            model.addCons(y[j] == 1, name=f"ForceOpen({j})")

    model.addCons(quicksum(y[j] for j in J) <= 6, "FacilityLimit") 
            
    model.setObjective(
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [5]:
# for I, d make a dictionary of planning units to number of students
pu_data = gpd.read_file('GIS_files/pu_with_proj.geojson').set_index('pu_2324_84')
pu_data = pu_data['final_proj'].to_dict()

I, d = multidict(pu_data)

In [6]:
# for J, M make a dictionary of sites to capacities
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/HS_regions')
schools = schools.to_crs('EPSG:4326')
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape.geojson')
pu = pu.to_crs('EPSG:4326')

# let's remove planning units in the North from J to make problem simpler
not_north = pu[(pu['Region'] != 'North') | (pu['pu_2324_848'] == 56)]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for _, row in not_north.iterrows():
    pu_dict[row['OBJECTID']] = 1600

# find which planning units have existing school
school_centroids = schools.geometry.centroid
schools['pu'] = None

for i, geometry in enumerate(pu['geometry']):
    in_geometry = school_centroids.within(geometry)
    pu_id = pu.loc[i, 'OBJECTID']
    schools.loc[in_geometry, 'pu'] = pu_id

# replace capacities of planning units with existing schools
pu_dict[56] = 1600
pu_dict[282] = 1810
pu_dict[591] = 1540
pu_dict[682] = 1540
pu_dict[779] = 1535

J, M = multidict(pu_dict)

# define which sites already exist
existing_sites = {56, 282, 591, 682, 779}

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_34824/950143076.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  school_centroids = schools.geometry.centroid


In [8]:
# create distance matrix
c = {}

school_centroids = schools.set_index('pu').geometry.centroid #use centroid of schools for distance calculation
pu_centroids = pu.set_index('OBJECTID').geometry.centroid 

for i in I:
    for j in J:
        dist = pu_centroids[i].distance(pu_centroids[j])
        c[i, j] = dist

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_34824/3496151326.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  school_centroids = schools.set_index('pu').geometry.centroid #use centroid of schools for distance calculation
/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_34824/3496151326.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pu_centroids = pu.set_index('OBJECTID').geometry.centroid


In [9]:
'''
for testing:
I_small = random.sample(I, 100)
d_small = {i: d[i] for i in I_small}
c_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}

model = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)
'''

presolving:
(round 1, fast)       12225 del vars, 12245 del conss, 0 add conss, 61105 chg bounds, 0 chg sides, 611 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, fast)       12225 del vars, 12650 del conss, 0 add conss, 61105 chg bounds, 0 chg sides, 611 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 3, fast)       12305 del vars, 12650 del conss, 0 add conss, 61105 chg bounds, 0 chg sides, 611 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 4, fast)       12305 del vars, 12650 del conss, 0 add conss, 61105 chg bounds, 80 chg sides, 611 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 5, fast)       33221 del vars, 33438 del conss, 0 add conss, 61105 chg bounds, 80 chg sides, 1217 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 6, exhaustive) 33221 del vars, 33438 del conss, 0 add conss, 61105 chg bounds, 80 chg sides, 1217 chg coeffs, 27705 upgd conss, 0 impls, 1 clqs
(round 7, exhaustive) 33348 del vars, 33438 del conss, 0 add conss, 61105 chg bounds, 80 chg sides, 1217 chg 

In [ ]:
model = flp(I, J, d, M, c, existing_sites=existing_sites)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)